In [ ]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Load your data
data = pd.read_csv('./data/joined_HPMD.csv')
df = pd.DataFrame(data)

# Remove years 2005 and 2006
df = df[~df["Year"].isin([2005, 2006])]

# Sort data by year to ensure ascending order
df = df.sort_values(by="Year")

# List of disorders for the dropdown
disorders = ["Schizophrenia", "Bipolar disorders", "Eating disorders",
             "Anxiety disorders", "Drug use disorders", "Depression", "Alcohol use disorders"]

# Initialize the Dash app
app = dash.Dash(__name__)

# Define a custom color scheme for 10 unique regions using "D3" palette
color_scheme = px.colors.qualitative.Pastel  # New color palette
unique_regions = df["Regional Indicator"].unique()
region_colors = {region: color_scheme[i % len(color_scheme)] for i, region in enumerate(unique_regions)}

# Dash Layout
app.layout = html.Div([
    # Dropdown for selecting disorder
    dcc.Dropdown(
        id="disorder-dropdown",
        options=[{"label": disorder, "value": disorder} for disorder in disorders],
        value="Schizophrenia",  # Initial value
        style={"width": "50%"}
    ),
    
    # Graph to display the scatter plot
    dcc.Graph(
        id="scatter-plot",
        style={"width": "90%", "height": "600px"}
    ),
])


@app.callback(
    Output("scatter-plot", "figure"),
    [Input("disorder-dropdown", "value")]
)
def update_scatter_plot(selected_disorder):
    # Calculate year-on-year change and take the absolute value
    df["Yearly_Change"] = df.groupby("Country Name")[selected_disorder].diff().fillna(0).abs()

    # Handle zero changes by replacing with a small value
    df["Yearly_Change"] = df["Yearly_Change"].replace(0, 0.01)

    # Scale the changes (min-max scaling with sqrt transformation)
    if df["Yearly_Change"].max() > 0:  # Avoid division by zero
        df["Scaled_Change"] = (df["Yearly_Change"] - df["Yearly_Change"].min()) / (
            df["Yearly_Change"].max() - df["Yearly_Change"].min()
        )
        df["Scaled_Change"] = (df["Scaled_Change"] ** 0.5) * 10000 + 50
    else:
        df["Scaled_Change"] = 50

    # Create scatter plot with disorder rate in hover data and animation frame for year
    fig = px.scatter(
        df,
        x="Log GDP Per Capita",
        y="Life Ladder",
        size="Scaled_Change",
        color="Regional Indicator",
        hover_data={
            "Country Name": True,
            "Log GDP Per Capita": True,
            "Life Ladder": True,
            "Yearly_Change": False,  # Hide scaled yearly change
            selected_disorder: True,  # Show the disorder rate
        },
        animation_frame="Year",  # Add animation frame for the timeline slider
        title=f"Happiness Score vs GDP with Scaled Yearly Change in {selected_disorder}",
        labels={"Scaled_Change": "Scaled Yearly Change in Disorder"},
        color_discrete_map=region_colors
    )
    
    # Manually set the axis range for x and y
    fig.update_layout(
        xaxis=dict(range=[5, 12]),  # Set fixed range for x-axis
        yaxis=dict(range=[2, 9]),   # Set fixed range for y-axis
        updatemenus=[{
            "buttons": [
                {"label": "Play", "method": "animate", "args": [None, {"frame": {"duration": 1000, "redraw": True}, "fromcurrent": True}]},
                {"label": "Pause", "method": "animate", "args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate", "fromcurrent": False}]}
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }]
    )
    return fig


# Run the Dash app
if __name__ == "__main__":
    app.run_server(debug=True, port=8054)

# open the website on http://127.0.0.1:8054


: 